In [1]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *
import re

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
db = client.adsE_database_tmp
collection = db.adsE_collection

In [5]:
def makejson(file):
    atoms = query(file, 'spacom')

    dic = {}

    res = re.match('(.*u\d).*', file)
    barefile = res.group(1) + '.traj'
    bareatoms = query(barefile, 'spacom')
#     print(bareatoms)

    COatoms = query('CO.traj', 'scpacom')
    COene = COatoms.get_potential_energy()

    res = re.match('(\w.*)_(\d{3}).*', file)
    ele = res.group(1)
    mir = res.group(2)
    
    num = 0
    ads = None
    res = re.match('.*n(\d+)_', file)
    if res:
        num = int(res.group(1))
        
    res = re.match('.*u(\d+).*', file)
    unit = int(res.group(1))

    res = re.match('.*no.*_(\w.)_n', file)
    if res:
        ads = res.group(1)

    ### calc area ###
    x = atoms.cell[0][0]
    y = atoms.cell[1][1]
    area = x*y

    ### calc adsorvates number ###
    tot = 0
    for atom in atoms:
        if atom.tag == 1:
            tot += 1

    ### get energies ###
    ene = atoms.get_potential_energy()
    bareene = bareatoms.get_potential_energy()
    totaladsene = ene - (COene*num + bareene)
    
    ### get adsorbate position info ###
    igroups, iposlis, rgroups, rposlis = getadsposinfo(file, 0)
    
    ### get COint info ###
    try:
        file_ = file[:-5] + '__.traj'
        atoms_ = query(file_, 'spacom')
        ene_ = atoms_.get_potential_energy()
    except:
        ene_ = 0
        
    ### get COlength info ###
    COlengthlis = []
    for i in range(len(atoms)):
        if atoms[i].symbol == 'C':
            Cpos = atoms[i].position
            Opos = atoms[i+1].position
            COlengthlis.append(np.linalg.norm(Cpos-Opos))
    
    ### get convergence ###
    converged = np.max(np.abs(atoms.get_forces())) < 0.03
        
    dic['name'] = file
    dic['element'] = ele
    dic['mirror'] = mir
    dic['adsorbate'] = ads
    dic['unitlength'] = unit
    dic['surfatomnum'] = tot
    dic['area'] = area
    dic['Energy'] = ene
    dic['totaladsEnergy'] = totaladsene
    dic['aveadsEne/suratom'] = totaladsene/tot
    dic['numberofads'] = num
    dic['coverage'] = num/tot
    dic['density'] = num/area
    dic['igroups'] = igroups
    dic['iposlis'] = [list(item) for item in iposlis]
    dic['rgroups'] = rgroups
    dic['rposlis'] = [list(item) for item in rposlis]
    dic['ECO_Eint'] = ene_
    dic['COlengthlis'] = COlengthlis
    dic['converged'] = 'yes' if converged else 'no'
    
    dic['aveadsEne/ads'] = totaladsene/num if num else None

    maxdiff = getmaxdiff(file)
    dic['moved+'] = 'yes' if maxdiff > 1.5 else 'no'
    dic['moved'] = 'no' if igroups == rgroups else 'yes'
    
    for key in dic.keys():
        print('{}: {}'.format(key, dic[key]))
    
    return dic

In [6]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')
for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if '__' not in files[i]:
                print(i, files[i])

7 Al_111_u2.traj
9 Al_111_u2_no001_CO_n1_d9.traj
12 Al_111_u2_no002_CO_n2_d3.traj
15 Al_111_u2_no003_CO_n3_d3.traj
18 Al_111_u2_no004_CO_n4_d3.traj
21 Al_111_u2_no005_CO_n2_d4.traj
24 Al_111_u2_no006_CO_n3_d4.traj
27 Al_111_u2_no007_CO_n2_d4.traj
30 Al_111_u2_no008_CO_n1_d9.traj
33 Al_111_u2_no009_CO_n2_d3.traj
36 Al_111_u2_no010_CO_n3_d3.traj
39 Al_111_u2_no011_CO_n4_d3.traj
42 Al_111_u2_no012_CO_n2_d3.traj
45 Al_111_u2_no013_CO_n2_d3.traj
48 Al_111_u2_no014_CO_n2_d3.traj
51 Al_111_u2_no015_CO_n1_d9.traj
54 Al_111_u2_no016_CO_n2_d4.traj
57 Al_111_u2_no017_CO_n2_d3.traj
60 Al_111_u2_no018_CO_n3_d3.traj
63 Al_111_u2_no019_CO_n4_d3.traj
66 Al_111_u2_no020_CO_n1_d9.traj
69 Al_111_u2_no021_CO_n2_d3.traj
72 Al_111_u2_no022_CO_n3_d3.traj
75 Al_111_u2_no023_CO_n4_d3.traj
86 CO.traj
92 Pt_111_u2.traj
94 Pt_111_u2_no01_CO_n1_d9.traj
97 Pt_111_u2_no02_CO_n2_d3.traj
100 Pt_111_u2_no03_CO_n3_d3.traj
103 Pt_111_u2_no04_CO_n4_d3.traj
106 Pt_111_u2_no05_CO_n2_d4.traj
109 Pt_111_u2_no06_CO_n3_d4.traj


In [7]:
# makejson(files[7])

In [8]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')

for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if 'CO.traj' not in files[i]:
                if '__' not in files[i]:
                    print(i, files[i])

                    post_data = makejson(files[i])
                    result = collection.insert_one(post_data)
                    print('One post: {0}\n'.format(result.inserted_id))

7 Al_111_u2.traj
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Al_111_u2.traj
element: Al
mirror: 111
adsorbate: None
unitlength: 2
surfatomnum: 4
area: 28.555431024941463
Energy: -53.33045309
totaladsEnergy: 0.0
aveadsEne/suratom: 0.0
numberofads: 0
coverage: 0.0
density: 0.0
igroups: []
iposlis: []
rgroups: []
rposli

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Al_111_u2_no006_CO_n3_d4.traj
element: Al
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 28.555431024941463
Energy: -97.027005
totaladsEnergy: -0.240242980000005
aveadsEne/suratom: -0.06006074500000125
numberofads: 3
coverage: 0.75
density: 0.10505882391968377

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Al_111_u2_no011_CO_n4_d3.traj
element: Al
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 28.555431024941463
Energy: -110.74201471
totaladsEnergy: 0.5301836200000025
aveadsEne/suratom: 0.13254590500000063
numberofads: 4
coverage: 1.0
density: 0.1400784318929117
igroups: [1, 1, 1, 1]
iposlis: [[6.4599858477862, 3.7296745681805357, 18.882741810078574], [2.1533286159287344, 1.2432248560601804, 18.882741810078574], [5.024433437167049, 1.2432248560601782, 18.882741810078574], [3.58888102654789, 3.729674568180535, 18.882741810078574]]
rgroups: [1, 1, 1, 1]
rposlis: [[6.480303383336732, 3.7414071966776947, 18.796210413061015], [2.173648122349312, 1.2549544882978525, 18.79621158774423], [5.044753106243125, 1.2549565255561868, 18.79621560232591], [3.609209912934554, 3.741412183798547, 18.796190013735913]]
ECO_Eint: -55.86190198
COlengthlis: [1.1803931164328014, 1.180356

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Al_111_u2_no017_CO_n2_d3.traj
element: Al
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 28.555431024941463
Energy: -82.5588249
totaladsEnergy: -0.2574991900000043
aveadsEne/suratom: -0.06437479750000108
numberofads: 2
coverage: 0.5
density: 0.0700392159464558

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Al_111_u2_no022_CO_n3_d3.traj
element: Al
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 28.555431024941463
Energy: -96.61837941
totaladsEnergy: 0.16838260999999477
aveadsEne/suratom: 0.04209565249999869
numberofads: 3
coverage: 0.75
density: 0.10505882391968377
igroups: [3, 3, 3]
iposlis: [[7.177762053095778, 4.144082853533927, 18.882741810078574], [2.8711048212383123, 1.6576331414135694, 18.882741810078574], [5.7422096424766265, 1.6576331414135694, 18.882741810078574]]
rgroups: [3, 3, 3]
rposlis: [[7.177672248827532, 4.142268999528871, 18.713222066415593], [2.869529229367987, 1.658437410864158, 18.713095821338726], [5.744850862065546, 1.6590933460117787, 18.71343742308509]]
ECO_Eint: -42.39864688
COlengthlis: [1.1899870393397414, 1.189886972153004, 1.1

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Pt_111_u2_no04_CO_n4_d3.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 27.585487923689545
Energy: -143.09795614
totaladsEnergy: -2.1700334500000054
aveadsEne/suratom: -0.5425083625000013
numberofads: 4
coverage: 1.0
density: 0.1450037791996032

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Pt_111_u2_no09_CO_n2_d3.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 27.585487923689545
Energy: -114.13751714
totaladsEnergy: -2.1804670699999917
aveadsEne/suratom: -0.5451167674999979
numberofads: 2
coverage: 0.5
density: 0.07250188959980161
igroups: [1, 1]
iposlis: [[3.527402647317733, 1.2219281207814556, 18.762269282616817], [0.7054805294635469, 1.2219281207814556, 18.762269282616817]]
rgroups: [1, 1]
rposlis: [[3.5571979004848275, 1.225359124513103, 18.488591949534804], [0.7352649789519946, 1.2253865411858484, 18.488571395181385]]
ECO_Eint: -28.55951815
COlengthlis: [1.1837148608899126, 1.1838812154679421]
converged: yes
aveadsEne/ads: -1.0902335349999959
moved+: no
moved: no
One post: 5de2de23687993289f68bf88

121 Pt_111_u2_no10_C

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Pt_111_u2_no15_CO_n1_d9.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 27.585487923689545
Energy: -98.86945491
totaladsEnergy: -1.397841150000005
aveadsEne/suratom: -0.34946028750000124
numberofads: 1
coverage: 0.25
density: 0.0362509447999008

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
name: Pt_111_u2_no20_CO_n1_d9.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 27.585487923689545
Energy: -98.86740252
totaladsEnergy: -1.395788760000002
aveadsEne/suratom: -0.3489471900000005
numberofads: 1
coverage: 0.25
density: 0.036250944799900806
igroups: [3]
iposlis: [[2.821922117854188, 1.6292374943752728, 18.762269282616817]]
rgroups: [3]
rposlis: [[2.821824309789016, 1.6291812404798118, 18.365339788888402]]
ECO_Eint: -14.48150749
COlengthlis: [1.199430878644389]
converged: yes
aveadsEne/ads: -1.395788760000002
moved+: no
moved: no
One post: 5de2de2a687993289f68bf93

154 Pt_111_u2_no21_CO_n2_d3.traj
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure fo